In [1]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV

In [3]:
# Load the data
scenarios_200 = CSV.read("../data/scenarios_200.csv", DataFrame)

# create a dictonary with 200 dataframes for each scenario
scenarios = Dict()
for i in 1:250
    df_helper = DataFrame(scenarios_200[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    #set column names
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    #change grid excess column to float
    
    scenarios[i] = df_helper
end

In [10]:
#show the last scenario
scenarios[250]

Row,price,wind power,grid_excess
,Float64,Float64,Float64
1,160.04,102.642,1.0
2,155.15,105.772,0.0
3,150.0,108.44,1.0
4,147.97,91.406,1.0
5,145.97,75.7806,0.0
6,149.1,61.3081,1.0
7,175.69,48.2785,1.0
8,196.64,35.3509,1.0
9,216.19,25.44,0.0


In [4]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[])

Row,hour,p_DA
,Int64,Float64


In [5]:
W = 250
hours = 24

24

In [6]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])

# Define the objective function
@objective(model, Max, sum(1/W .*
(scenarios[i][hour,"price"] * p_DA[hour]
+ scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])


# Solve the optimization problem
optimize!(model)


#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found")
end

Optimal solution found
Objective value: 150962.34995728184


In [7]:
println("Objective value: ", objective_value(model))
println("p_DA: ", value.(p_DA))
println("delta: ", value.(delta))

Objective value: 150962.34995728184
p_DA: [0.0, 0.0, 200.0, 200.0, 200.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 0.0, 0.0, 0.0, 200.0]
delta: [151.99241932258064 143.45919354838708 -57.41322580645161 -53.381612919354836 -49.864032306451634 151.10161283870966 -54.073064548387094 143.26112901612905 142.27870962903225 -57.78467748387095 141.57870962903226 138.31774191935483 -62.66483872580645 136.8064515967742 -67.00290325806449 129.52919346774195 -78.1700000483871 -84.87403232258065 108.50145154838708 -96.45903225806452 98.1822580483871 92.16596767741936 84.6527419032258 -121.93048388709677; 22.33724183870968 18.609870870967743 -184.0500645483871 -186.53041943548388 -190.75709683870969 5.90827414516129 -195.4805161451613 2.7339112741935483 1.1038967741935486 -199.02258872580646 0.7919354838709678 0.7419306451612904 -199.0258064516129 1.167741935483871 -198.67265 2.3072451612903224 -196.67336451612903 -195.50001612903225 5.493548387096774 

In [12]:
# make a table with the results
for hour in 1:hours
    push!(result_df, [hour, value(p_DA[hour])])
end

result_df

Row,hour,p_DA
,Int64,Float64
1,1,0.0
2,2,0.0
3,3,200.0
4,4,200.0
5,5,200.0
6,6,0.0
7,7,200.0
8,8,0.0
9,9,0.0
